In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv(r'/kaggle/input/bike-share-daily-data/bike_sharing_daily.csv')
df.set_index('dteday', inplace=True)
df.drop('instant', axis=1, inplace=True)
print(df.shape)
df.head()

In [ ]:
df['cnt'].plot(figsize=(20,5))

In [ ]:
import scipy.signal
df['cnt'] = scipy.signal.detrend(df['cnt'])
df['cnt'].plot(figsize=(20,5))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, Normalizer, MinMaxScaler 
ct = ColumnTransformer([
    ("onehot", OneHotEncoder(categories='auto'), ['season', 'mnth', 'weekday', 'weathersit']),
    ("pass", "passthrough", ['holiday', 'workingday', 'temp', 'atemp', 'hum']),
    ("drop", "drop", ['yr', 'casual', 'registered', 'cnt']),
    ("norm", MinMaxScaler(), ['windspeed']),
])
X1 = ct.fit_transform(df)
X1.shape

In [ ]:
from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.models import model_from_json

act_func = 'elu'

model = Sequential()
model.add(Dense(64, activation=act_func, input_shape=(X1.shape[1],)))
model.add(Dense(32, activation=act_func))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse',optimizer='rmsprop',metrics=['mse'])
model.summary()


In [ ]:
NUM_EPOCHS=100
BATCH_SIZE=10

history=model.fit(
    X1,np.array(df['cnt']),
    batch_size=BATCH_SIZE, 
    epochs=NUM_EPOCHS,
    validation_split=0.05,
    verbose = 1,
)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['loss'],
         'b',
         label='Training loss')
plt.plot(history.history['val_loss'],
         'r',
         label='Validation loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')
#plt.ylim([0,.1])
plt.show()

In [ ]:
X_pred = model.predict(X1)
result = pd.DataFrame(X_pred)
result.index = df.index
result['actual'] = df['cnt']
result.plot(figsize=(20,5))
